In [1]:
import numpy as np
import pandas as pd

from keras.utils import multi_gpu_model
from keras.preprocessing.image import ImageDataGenerator
from utils import load_pretrained_weights
from preprocessing import center_crop

Using TensorFlow backend.


In [3]:
processed_imgs_dir = "/mnt/safe01/data/processed/"
train_dir = "/mnt/safe01/data/train/"
validation_dir = "/mnt/safe01/data/validation/"
test_dir = "/mnt/safe01/data/test/"
weights_path = "/home/nhannguyen/Kaggle_IEEE/model_weights/inception_resnet_v2/"

In [5]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True)
train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    batch_size=36,
                                                    target_size=(299, 299))

Found 147060 images belonging to 10 classes.


In [33]:
valid_datagen = ImageDataGenerator(rescale=1. / 255,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True)
valid_generator = valid_datagen.flow_from_directory(validation_dir, 
                                                    batch_size=36,
                                                    target_size=(299, 299))

Found 1440 images belonging to 10 classes.


In [7]:
inceptionresnetv2 = load_pretrained_weights('InceptionResNetV2', input_shape=(299, 299, 3))

In [34]:
inceptionresnetv2.load_weights(weights_path + 'multigpu7.hdf5')

In [35]:
parallel_model = multi_gpu_model(model=inceptionresnetv2, gpus=4)

In [26]:
def predict_image_generator(path, model, batch_size, num_test_samples, class_indices):
    test_datagen = ImageDataGenerator(rescale=1. / 255)
    test_generator = test_datagen.flow_from_directory(path,
                                                      batch_size=batch_size,
                                                      target_size=(299, 299),
                                                      shuffle=False)
    y_pred = model.predict_generator(test_generator, steps=num_test_samples / batch_size,
                                     verbose=1)
    indices_pred = np.argmax(y_pred, axis=1)
    indices_class = {v: k for k, v in class_indices.items()}
    class_pred = np.vectorize(indices_class.get)(indices_pred)
    return class_pred

class_pred = predict_image_generator(path=test_dir, model=parallel_model,
                                     batch_size=40, num_test_samples=2640,
                                     class_indices=train_generator.class_indices)

Found 2640 images belonging to 1 classes.
66/66 [==============================] - 89s 1s/step


In [31]:
def make_submission(input_path, output_path, camera_prediction):
    """
    Concatenate the test images names with their associated camera prediction in a single
    DataFrame.
    """
    import os
    all_test_img_names = sorted(os.listdir(input_path))
    all_test_img_renamed = pd.Series([fname.replace('.jpg', '.tif') for 
                                      fname in all_test_img_names])
    camera = pd.Series(camera_prediction)
    df = pd.concat([all_test_img_renamed, camera], axis=1)
    df.columns = ["fname", "camera"]
    df.to_csv(output_path, index=False)
    return df

make_submission(input_path=test_dir+'cropped_test',
                output_path="/home/nhannguyen/Kaggle_IEEE/submission.csv",
                camera_prediction=class_pred).head()

,fname,camera
0,img_0002a04_manip.tif,Samsung-Galaxy-S4
1,img_001e31c_unalt.tif,iPhone-6
2,img_00275cf_manip.tif,iPhone-6
3,img_0034113_unalt.tif,Samsung-Galaxy-S4
4,img_00344b7_unalt.tif,Samsung-Galaxy-Note3
